In [1]:
import math
from decimal import *
import numpy as np # linear algebra
import pandas as pd # read_csv and such
from geopy.distance import vincenty # calculate distance between locations
from io import StringIO # convert strings to buffers or something like that.
import os # for listing files in directory

pd.options.display.max_columns = 999

In [2]:
# the paths to our flight related data sets
airlines_path = '../data/flight_delays_2015/airlines.csv'
airports_path = '../data/flight_delays_2015/airports.csv'
flights_path  = '../data/flight_delays_2015/flights.csv'
# weather related paths
weather_path = '../data/weather/2015.csv'
stations_path  = '../data/weather/stations.csv'
weather_files_path = '../data/weather/station_data/'
weather_merged_path = '../data/weather/weather_merged.csv'

# modified data sets for output
airports_modified_path = '../data/flight_delays_2015/airports_modified.csv'
flights_weather_path = '../data/flight_delays_2015/flights_weather.csv'

In [3]:
dtypes = {
    'ORIGIN_AIRPORT': 'str', 
    'DESTINATION_AIRPORT': 'str', 
    'IATA_CODE_x': 'str', 
    'origin_weather_station': 'str', 
    'IATA_CODE_y': 'str', 
    'destination_weather_station': 'str', 
    'OR_MAX': 'str', 
    'OR_MIN': 'str', 
    'OR_PRCP': 'str', 
    'DES_MAX': 'str', 
    'DES_MIN': 'str', 
    'DES_PRCP': 'str', 
    'OR_FRSHTT': 'str', 
    'DES_FRSHTT': 'str'
}
fw_df = pd.read_csv(flights_weather_path, dtype=dtypes, parse_dates=['DATE'])
fw_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,OR_TEMP,OR_COUNT_TEMP,OR_DEWP,OR_COUNT_DEWP,OR_SLP,OR_COUNT_SLP,OR_STP,OR_COUNT_STP,OR_VISIB,OR_COUNT_VISIB,OR_WDSP,OR_COUNT_WDSP,OR_MXSPD,OR_GUST,OR_MAX,OR_MIN,OR_PRCP,OR_SNDP,OR_FRSHTT,DES_TEMP,DES_COUNT_TEMP,DES_DEWP,DES_COUNT_DEWP,DES_SLP,DES_COUNT_SLP,DES_STP,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_MAX,DES_MIN,DES_PRCP,DES_SNDP,DES_FRSHTT,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,ANC,702725-26491,SEA,727930-24233,35.1,24,30.7,24,1020.1,14,1015.2,24,7.5,24,3.6,24,6.0,999.9,43.0,32.0,0.09G,999.9,110000,32.9,24,22.2,24,1029.7,24,1012.7,24,10.0,24,4.6,24,11.1,999.9,42.1,26.1,0.00G,999.9,000000,True,True,False,False,False,False,False,False,False,False,False,False
1,2015,1,1,4,AS,108,N309AS,ANC,SEA,45,41.0,-4.0,17.0,58.0,204.0,194.0,173.0,1448,451.0,4.0,509,455.0,-14.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,ANC,702725-26491,SEA,727930-24233,35.1,24,30.7,24,1020.1,14,1015.2,24,7.5,24,3.6,24,6.0,999.9,43.0,32.0,0.09G,999.9,110000,32.9,24,22.2,24,1029.7,24,1012.7,24,10.0,24,4.6,24,11.1,999.9,42.1,26.1,0.00G,999.9,000000,True,True,False,False,False,False,False,False,False,False,False,False
2,2015,1,1,4,DL,1560,N3743H,ANC,SEA,45,31.0,-14.0,25.0,56.0,210.0,200.0,171.0,1448,447.0,4.0,515,451.0,-24.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,ANC,702725-26491,SEA,727930-24233,35.1,24,30.7,24,1020.1,14,1015.2,24,7.5,24,3.6,24,6.0,999.9,43.0,32.0,0.09G,999.9,110000,32.9,24,22.2,24,1029.7,24,1012.7,24,10.0,24,4.6,24,11.1,999.9,42.1,26.1,0.00G,999.9,000000,True,True,False,False,False,False,False,False,False,False,False,False
3,2015,1,1,4,AS,136,N431AS,ANC,SEA,135,NaN,NaN,NaN,NaN,205.0,NaN,NaN,1448,NaN,NaN,600,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-01-01,ANC,702725-26491,SEA,727930-24233,35.1,24,30.7,24,1020.1,14,1015.2,24,7.5,24,3.6,24,6.0,999.9,43.0,32.0,0.09G,999.9,110000,32.9,24,22.2,24,1029.7,24,1012.7,24,10.0,24,4.6,24,11.1,999.9,42.1,26.1,0.00G,999.9,000000,True,True,False,False,False,False,False,False,False,False,False,False
4,2015,1,1,4,AS,134,N464AS,ANC,SEA,155,140.0,-15.0,17.0,157.0,218.0,198.0,170.0,1448,547.0,11.0,633,558.0,-35.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,ANC,702725-26491,SEA,727930-24233,35.1,24,30.7,24,1020.1,14,1015.2,24,7.5,24,3.6,24,6.0,999.9,43.0,32.0,0.09G,999.9,110000,32.9,24,22.2,24,1029.7,24,1012.7,24,10.0,24,4.6,24,11.1,999.9,42.1,26.1,0.00G,999.9,000000,True,True,False,False,False,False,False,False,False,False,False,False


Convert some of these columns to numeric data and remove values that indicated as missing.

In [4]:
fw_df['OR_PRCP'] = pd.to_numeric(fw_df['OR_PRCP'].str.replace('A|B|C|D|E|F|G|H|I', ''))
fw_df['OR_MIN'] = pd.to_numeric(fw_df['OR_MIN'].str.replace('*', ''))
fw_df['OR_MAX'] = pd.to_numeric(fw_df['OR_MAX'].str.replace('*', ''))

In [5]:
fw_df.loc[fw_df['OR_TEMP'] == 9999.9, 'OR_TEMP'] = np.NaN
fw_df.loc[fw_df['OR_MIN'] == 9999.9, 'OR_MIN'] = np.NaN
fw_df.loc[fw_df['OR_MAX'] == 9999.9, 'OR_MAX'] = np.NaN
fw_df.loc[fw_df['OR_WDSP'] == 999.9, 'OR_WDSP'] = np.NaN
fw_df.loc[fw_df['OR_MXSPD'] == 999.9, 'OR_MXSPD'] = np.NaN
fw_df.loc[fw_df['OR_GUST'] == 999.9, 'OR_GUST'] = np.NaN
fw_df.loc[fw_df['OR_SNDP'] == 999.9, 'OR_SNDP'] = np.NaN
fw_df.loc[fw_df['OR_PRCP'] == 99.99, 'OR_PRCP'] = np.NaN

### Create datafames that contain where flights are only weather delayed at origin or at destination
This will not contain all delays that fit these categories but those that we can be fairly certain fit a category.

Find destination only delays by checking where the flight departed on time, but arrived late and has a weather delay.

In [6]:
dest_delay_df = fw_df[(fw_df['WEATHER_DELAY'] > 0) & (fw_df['DEPARTURE_DELAY'] < 0) & (fw_df['ARRIVAL_DELAY'] > 0)]
dest_delay_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,OR_TEMP,OR_COUNT_TEMP,OR_DEWP,OR_COUNT_DEWP,OR_SLP,OR_COUNT_SLP,OR_STP,OR_COUNT_STP,OR_VISIB,OR_COUNT_VISIB,OR_WDSP,OR_COUNT_WDSP,OR_MXSPD,OR_GUST,OR_MAX,OR_MIN,OR_PRCP,OR_SNDP,OR_FRSHTT,DES_TEMP,DES_COUNT_TEMP,DES_DEWP,DES_COUNT_DEWP,DES_SLP,DES_COUNT_SLP,DES_STP,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_MAX,DES_MIN,DES_PRCP,DES_SNDP,DES_FRSHTT,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD
8318,2015,4,15,3,VX,781,N524VA,LAX,SEA,730,729.0,-1.0,9.0,738.0,155.0,177.0,157.0,954,1015.0,11.0,1005,1026.0,21.0,0,0,NaN,0.0,0.0,0.0,0.0,21.0,2015-04-15,LAX,722950-23174,SEA,727930-24233,63.7,24,48.2,24,1014.4,24,1002.6,24,9.9,24,6.5,24,17.1,NaN,75.0,55.9,0.0,NaN,000000,45.3,24,36.8,24,1033.0,24,1016.1,24,10.0,24,4.8,24,8.0,999.9,57.0,37.0,0.13G,999.9,010000,False,False,False,False,False,False,False,True,False,False,False,False
9549,2015,6,5,5,VX,781,N837VA,LAX,SEA,745,743.0,-2.0,9.0,752.0,150.0,168.0,146.0,954,1018.0,13.0,1015,1031.0,16.0,0,0,NaN,0.0,0.0,0.0,0.0,16.0,2015-06-05,LAX,722950-23174,SEA,727930-24233,62.7,24,55.3,24,1011.1,22,999.4,24,10.0,24,7.4,24,15.9,22.0,68.0,59.0,0.0,NaN,000000,66.2,24,48.4,24,1020.4,24,1004.1,24,10.0,24,8.3,24,17.1,24.1,80.1,53.1,0.00G,999.9,000000,False,False,False,False,False,False,False,False,False,False,False,False
11037,2015,8,4,2,VX,781,N852VA,LAX,SEA,745,744.0,-1.0,10.0,754.0,150.0,167.0,145.0,954,1019.0,12.0,1015,1031.0,16.0,0,0,NaN,0.0,0.0,0.0,0.0,16.0,2015-08-04,LAX,722950-23174,SEA,727930-24233,72.2,24,63.0,24,1012.1,24,1000.6,24,9.9,24,6.9,24,18.1,22.9,80.1,66.0,0.0,NaN,000000,67.6,24,54.0,24,1015.5,23,999.4,24,10.0,24,5.1,24,9.9,14.0,82.0*,57.2*,0.00G,999.9,000000,False,False,False,False,False,False,False,False,False,False,False,False
11822,2015,9,4,5,VX,781,N524VA,LAX,SEA,800,759.0,-1.0,16.0,815.0,150.0,176.0,147.0,954,1042.0,13.0,1030,1055.0,25.0,0,0,NaN,0.0,0.0,0.0,0.0,25.0,2015-09-04,LAX,722950-23174,SEA,727930-24233,69.8,24,57.3,24,1011.0,23,999.5,24,10.0,24,6.4,24,13.0,NaN,75.0,66.9,0.0,NaN,000000,55.7,24,47.2,24,1016.7,20,1000.3,24,10.0,24,6.0,24,8.9,999.9,64.9,50.0,0.00G,999.9,000000,False,False,False,False,False,False,False,False,False,False,False,False
16164,2015,4,14,2,VX,740,N521VA,SFO,SEA,800,755.0,-5.0,13.0,808.0,120.0,140.0,118.0,679,1006.0,9.0,1000,1015.0,15.0,0,0,NaN,0.0,0.0,0.0,0.0,15.0,2015-04-14,SFO,724940-23234,SEA,727930-24233,55.8,24,41.3,24,1024.2,24,1023.3,24,10.0,24,12.4,24,24.1,27.0,63.0,50.0,0.0,NaN,000000,42.3,24,36.6,24,1022.2,17,1004.4,24,8.7,24,7.0,24,14.0,15.9,53.1,37.0,0.55G,999.9,010000,False,False,False,False,False,False,False,True,False,False,False,False


Find origin delays by checking that there is a weather delay, the delay at arrival is no more than departure and there is a departure delay.

In [7]:
origin_delay_df = fw_df[(fw_df['WEATHER_DELAY'] > 0) & (fw_df['ARRIVAL_DELAY'] <= fw_df['DEPARTURE_DELAY']) & (fw_df['DEPARTURE_DELAY'] > 0)]
origin_delay_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,OR_TEMP,OR_COUNT_TEMP,OR_DEWP,OR_COUNT_DEWP,OR_SLP,OR_COUNT_SLP,OR_STP,OR_COUNT_STP,OR_VISIB,OR_COUNT_VISIB,OR_WDSP,OR_COUNT_WDSP,OR_MXSPD,OR_GUST,OR_MAX,OR_MIN,OR_PRCP,OR_SNDP,OR_FRSHTT,DES_TEMP,DES_COUNT_TEMP,DES_DEWP,DES_COUNT_DEWP,DES_SLP,DES_COUNT_SLP,DES_STP,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_MAX,DES_MIN,DES_PRCP,DES_SNDP,DES_FRSHTT,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD
5580,2015,11,10,2,AS,108,N319AS,ANC,SEA,45,126.0,41.0,15.0,141.0,201.0,200.0,179.0,1448,540.0,6.0,506,546.0,40.0,0,0,NaN,0.0,0.0,0.0,0.0,40.0,2015-11-10,ANC,702725-26491,SEA,727930-24233,31.1,24,27.7,24,995.6,17,990.9,24,8.8,24,2.2,24,7.0,NaN,39.0,28.0,0.02,NaN,111000,44.9,24,41.7,24,1022.6,12,1005.8,24,6.9,24,4.1,24,8.9,999.9,52.0,39.0,0.13G,999.9,110000,True,True,True,False,False,False,True,True,False,False,False,False
5851,2015,11,28,6,DL,822,N372DA,ANC,SEA,50,134.0,44.0,12.0,146.0,205.0,201.0,182.0,1448,548.0,7.0,515,555.0,40.0,0,0,NaN,0.0,0.0,0.0,0.0,40.0,2015-11-28,ANC,702725-26491,SEA,727930-24233,35.5,24,31.1,24,997.1,24,992.4,24,10.0,24,2.3,24,7.0,17.1,45.0,32.0,0.01,NaN,000000,35.8,24,29.2,24,1027.5,24,1010.6,24,10.0,24,3.0,24,9.9,999.9,48.9,27.0,0.00G,999.9,000000,False,False,False,False,False,False,False,False,False,False,False,False
6900,2015,1,31,6,OO,5608,N789SK,LAX,SEA,840,928.0,48.0,10.0,938.0,164.0,160.0,143.0,954,1201.0,7.0,1124,1208.0,44.0,0,0,NaN,0.0,0.0,0.0,9.0,35.0,2015-01-31,LAX,722950-23174,SEA,727930-24233,60.0,24,55.1,24,1011.5,22,999.9,24,8.5,24,3.5,24,12.0,NaN,68.0,55.0,0.04,NaN,010000,39.7,24,37.8,24,1024.9,18,1008.3,24,1.5,24,2.8,24,7.0,999.9,46.9,34.0,0.00G,999.9,100000,False,True,False,False,False,False,True,False,False,False,False,False
7674,2015,3,15,7,AS,449,N795AS,LAX,SEA,1350,1457.0,67.0,20.0,1517.0,167.0,153.0,126.0,954,1723.0,7.0,1637,1730.0,53.0,0,0,NaN,0.0,0.0,0.0,0.0,53.0,2015-03-15,LAX,722950-23174,SEA,727930-24233,74.9,24,40.9,24,1016.1,24,1004.4,24,10.0,24,5.0,24,12.0,NaN,90.0,64.0,0.00,NaN,000000,49.2,24,45.4,24,1011.0,21,995.3,24,6.4,24,6.9,24,15.9,26.0,57.0,46.0,0.42G,999.9,010000,False,False,False,False,False,False,False,True,False,False,False,False
7932,2015,3,28,6,OO,5608,N919SW,LAX,SEA,920,1021.0,61.0,19.0,1040.0,175.0,156.0,132.0,954,1252.0,5.0,1215,1257.0,42.0,0,0,NaN,0.0,0.0,0.0,0.0,42.0,2015-03-28,LAX,722950-23174,SEA,727930-24233,64.0,24,56.5,24,1017.2,23,1005.7,24,8.2,24,5.4,24,11.1,NaN,82.9,57.0,0.00,NaN,100000,53.7,24,45.4,24,1021.8,23,1005.1,24,10.0,24,13.1,24,18.1,25.1,71.1,48.0,0.04G,999.9,010000,True,False,False,False,False,False,False,True,False,False,False,False


In [8]:
both_delay_df = fw_df[(fw_df['WEATHER_DELAY'] > 0) & (fw_df['ARRIVAL_DELAY'] > fw_df['DEPARTURE_DELAY']) & (fw_df['DEPARTURE_DELAY'] > 0) & ~(fw_df['AIR_SYSTEM_DELAY'] > 0) & ~(fw_df['SECURITY_DELAY'] > 0) & ~(fw_df['AIRLINE_DELAY'] > 0) & ~(fw_df['LATE_AIRCRAFT_DELAY'] > 0)]
both_delay_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,OR_TEMP,OR_COUNT_TEMP,OR_DEWP,OR_COUNT_DEWP,OR_SLP,OR_COUNT_SLP,OR_STP,OR_COUNT_STP,OR_VISIB,OR_COUNT_VISIB,OR_WDSP,OR_COUNT_WDSP,OR_MXSPD,OR_GUST,OR_MAX,OR_MIN,OR_PRCP,OR_SNDP,OR_FRSHTT,DES_TEMP,DES_COUNT_TEMP,DES_DEWP,DES_COUNT_DEWP,DES_SLP,DES_COUNT_SLP,DES_STP,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_MAX,DES_MIN,DES_PRCP,DES_SNDP,DES_FRSHTT,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD
136564,2015,9,3,4,HA,26,N390HA,HNL,PDX,1405,1502.0,57.0,33.0,1535.0,330.0,350.0,309.0,2603,2344.0,8.0,2235,2352.0,77.0,0,0,NaN,0.0,0.0,0.0,0.0,77.0,2015-09-03,HNL,911820-22521,PDX,726980-24229,80.9,24,75.4,24,1011.0,22,1010.7,24,9.6,24,5.1,24,8.9,18.1,87.1,75.0,0.17,NaN,010010,60.6,24,48.5,24,1015.6,24,1012.0,24,10.0,24,3.8,24,11.1,999.9,73.0,50.0,0.05G,999.9,010000,False,True,False,False,True,False,False,True,False,False,False,False
453853,2015,11,20,5,HA,21,N384HA,SEA,HNL,835,843.0,8.0,31.0,914.0,380.0,391.0,353.0,2677,1307.0,7.0,1255,1314.0,19.0,0,0,NaN,0.0,0.0,0.0,0.0,19.0,2015-11-20,SEA,727930-24233,HNL,911820-22521,39.1,24,33.1,24,1031.1,23,1014.4,24,7.6,24,6.5,24,12.0,NaN,48.0,33.1,0.01,NaN,100000,75.4,24,72.5,24,1011.9,22,1011.5,24,8.9,24,4.9,24,12.0,999.9,86.0,73.0,0.29G,999.9,010000,True,False,False,False,False,False,False,True,False,False,False,False
466913,2015,9,3,4,HA,365,N486HA,OGG,HNL,1400,1427.0,27.0,5.0,1432.0,37.0,66.0,37.0,100,1509.0,24.0,1437,1533.0,56.0,0,0,NaN,0.0,0.0,0.0,0.0,56.0,2015-09-03,OGG,911900-22516,HNL,911820-22521,81.8,24,72.7,24,1012.2,24,1010.1,24,10.0,24,5.6,24,15.0,18.1,90.0,73.4,0.06,NaN,010010,80.9,24,75.4,24,1011.0,22,1010.7,24,9.6,24,5.1,24,8.9,18.1,87.1,75.0,0.17G,999.9,010010,False,True,False,False,True,False,False,True,False,False,True,False
470493,2015,2,14,6,HA,119,N492HA,KOA,HNL,1004,1110.0,66.0,7.0,1117.0,44.0,56.0,35.0,163,1152.0,14.0,1048,1206.0,78.0,0,0,NaN,0.0,0.0,0.0,0.0,78.0,2015-02-14,KOA,911975-21510,HNL,911820-22521,74.9,24,68.0,24,1007.9,24,1006.4,24,10.0,24,19.5,24,27.0,33.0,86.0,69.1,0.00,NaN,000000,77.3,24,71.7,24,1005.8,24,1005.3,24,9.2,24,14.7,24,22.9,33.0,82.0,73.0,0.05G,999.9,010000,False,False,False,False,False,False,False,True,False,False,False,False
474731,2015,9,15,2,HA,279,N490HA,KOA,HNL,1746,1751.0,5.0,22.0,1813.0,44.0,61.0,33.0,163,1846.0,6.0,1830,1852.0,22.0,0,0,NaN,0.0,0.0,0.0,0.0,22.0,2015-09-15,KOA,911975-21510,HNL,911820-22521,84.4,24,72.9,24,1012.0,24,1010.4,24,10.0,24,6.7,24,11.1,NaN,90.0,78.1,0.00,NaN,000000,81.3,24,74.2,24,1013.5,20,1012.9,24,9.3,24,7.8,24,15.9,22.9,89.1,77.0,0.35G,999.9,010000,False,False,False,False,False,False,False,True,False,False,False,False


Since the dataframe with flights that have origin based weather delays are the only one that has significant number of entries, let's focus some basic analysis on these.

First, let's check the mean of the weather values relative to the mean of all the flights.

In [13]:
fw_df[['OR_TEMP', 'OR_MIN', 'OR_MAX', 'OR_WDSP', 'OR_MXSPD', 'OR_GUST', 'OR_PRCP', 'OR_SNDP']].mean()

OR_TEMP     61.333791
OR_MIN      51.190735
OR_MAX      70.381713
OR_WDSP      7.025933
OR_MXSPD    14.014208
OR_GUST     23.073898
OR_PRCP      0.111353
OR_SNDP      4.516190
dtype: float64

In [12]:
origin_delay_df[['OR_TEMP', 'OR_MIN', 'OR_MAX', 'OR_WDSP', 'OR_MXSPD', 'OR_GUST', 'OR_PRCP', 'OR_SNDP']].mean()

OR_TEMP     60.072583
OR_MIN      51.212050
OR_MAX      69.288994
OR_WDSP      7.691552
OR_MXSPD    17.057134
OR_GUST     26.882647
OR_PRCP      0.252017
OR_SNDP      5.227842
dtype: float64

At first glance the only numbers that indicate a large difference are precipitation which more than doubles and snow depth which increases by less about 15%. Other numbers that may indicate correlations are max windspeed and gust.